In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '18'
plt.rcParams['figure.dpi'] = 125
plt.rcParams['figure.facecolor'] = 'white'

import numpy as np
import xarray as xr
import os, sys

# from pysst import SST

import os
import sys  
module_path = os.path.abspath(os.path.join('/Users/gilaverbuch/dev/pysst'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.pysst.pysst import SST



In [ ]:
date_i = '2020-06-10T12:00'
date_f = '2020-06-11T12:00'

date_i = np.datetime64(date_i)
date_f = np.datetime64(date_f)

date_i = np.datetime64('2023-07-26T16:11:00')

lat_min = 39
lat_max = 42
lon_min = -73 
lon_max = -70

s = SST(date_i=date_i)

In [ ]:
s.load_data()

In [ ]:
s.merge()

In [ ]:
s.sst

In [ ]:
s.sst.mcsst.plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
s.sst.mcsst_source1[6].plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
s.sst.mcsst_source2[6].plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
ds2 = nc.open_thredds('http://basin.ceoe.udel.edu/thredds/dodsC/avhrr_unfiltered_sst.nc')
dsx2 = ds2.to_xarray()
dsx2 = dsx2.sortby('time')
date_i = np.datetime64('2023-07-26T16:11:00')
dsx2 = dsx2.sel(time=date_i, method='nearest')

In [ ]:
dsx2.mcsst.plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
name = 'https://tds.maracoos.org/thredds/dodsC/AVHRR1.nc'
# name = 'https://tds.maracoos.org/thredds/dodsC/AVHRR.nc'
ds1 = nc.open_thredds(name)

In [ ]:
dsx1 = ds1.to_xarray()

In [ ]:
dsx1.mcsst[6].plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
dsx1.mcsst[6].plot.pcolormesh(vmin=0, vmax=20, cmap='gnuplot2')

In [ ]:
import nctoolkit as nc

In [ ]:
name = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/pathfinder/Version5.3/L3C/2023/data/20230105225822-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA19_G_2023005_day-v02.0-fv01.0.nc'
ds1 = nc.open_thredds(name)

In [ ]:
lat_min = 50 
lat_max = 30 
lon_min = -80 
lon_max = -60
dsx1 = ds1.to_xarray()
dsx1 = dsx1.sel(lat=slice(lat_min, lat_max))
dsx1 = dsx1.sel(lon=slice(lon_min, lon_max))

In [ ]:
dsx1

In [ ]:
dsx1.load()

In [ ]:
dsx1.sea_surface_temperature.plot(vmin=265, vmax=300, cmap='gnuplot2')

In [ ]:
dsx1

In [ ]:
nc.open_data("https://www.ncei.noaa.gov/thredds-ocean/dodsC/pathfinder/Version5.3/L3C/2023/data/20230101023227-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA19_G_2023001_night-v02.0-fv01.0.nc")